# Portfolio VaR Backtesting Simulation

This notebook simulates portfolio log returns and compares three parametric Value-at-Risk (VaR) approaches.  
It then backtests these models with 1-day and 10-day horizons (both overlapping and non-overlapping) using Kupiec tests.

---

## User Parameters

At the top of the script you can adjust:

- `NUM_ASSETS` : number of assets  
- `NUM_DAYS` : sample length  
- `CONF_LEVEL` : confidence level (e.g. 0.99)  
- `HOLDING_PERIOD` : horizon in days (e.g. 10)  
- `WINDOW_TYPE` : covariance estimation ("rolling" or "ema")  
- `WINDOW` / `DECAY` : window size or decay factor  
- `CORR_REGIME` : correlation structure ("high", "medium", "low")  
- `N_CLUSTERS` : number of correlation clusters  
- `VOL_REGIME` : volatility regime ("high", "medium", "low")  
- `PORTFOLIO_MIX` : "fixed" (static weights) or "dynamic" (time-varying)  
- `ALLOW_SHORTS` : allow individual short positions  
- `NET_BIAS` : overall net tilt: +1 = long, 0 = neutral, -1 = short  

**Note:** Portfolio weights are always scaled so that gross exposure = 1  
(sum of absolute weights = 1). VaR is reported as % of portfolio value.

---

## VaR Methods (mean = 0 assumed)

| Method         | Assumption                        | Formula (h = horizon, σ = volatility, z = quantile)        | VaR reported |
|----------------|-----------------------------------|------------------------------------------------------------|--------------|
| Normal-Simple  | Simple returns ~ Normal           | VaR = - z · σ · √h                                         | Simple %     |
| Lognormal      | Log returns ~ Normal              | VaR = - ( exp(z · σ · √h) - 1 )                            | Simple %     |
| Normal-Log     | Log returns ~ Normal              | VaR = - z · σ · √h                                         | Log returns → converted to simple % for backtesting |

**Note:** Although Normal-Log is the most statistically accurate in log space, for backtesting we care about exceedances in *portfolio value (% terms)*.  
Therefore Lognormal VaR is the most consistent with both the data-generating process and the exceedance criterion.

---

## Backtesting

- Realized portfolio P&L is always converted into **simple % returns**.  
- All three VaR estimates are expressed in **simple % terms** before exceedance checks.  
- This ensures a fair and consistent comparison: exceedances mean "portfolio value dropped below VaR by more than X%".  

Expected performance:  
- Lognormal VaR should be best calibrated.  
- Normal-Log close at short horizons but less consistent when converted to % terms.  
- Normal-Simple weakest, especially at longer horizons or higher volatility.

In [11]:
import numpy as np
import pandas as pd
from scipy.stats import norm, chi2

# -------------------------
# USER PARAMETERS (TOGGLES)
# -------------------------
NUM_ASSETS     = 5
NUM_DAYS       = 10_000
CONF_LEVEL     = 0.99
HOLDING_PERIOD = 10        # scaling horizon
WINDOW_TYPE    = "rolling"     # "rolling" or "ema"
WINDOW         = 250
DECAY          = 0.94

CORR_REGIME    = "medium"  # "high", "medium", "low"
N_CLUSTERS     = 3         # number of correlation clusters
VOL_REGIME     = "medium"  # "high", "medium", "low"

PORTFOLIO_MIX  = "fixed" # "fixed" or "dynamic"
ALLOW_SHORTS   = True      # allow individual assets to be short?

# NET_BIAS controls the overall tilt of the portfolio:
#   +1.0 → net long
#    0.0 → market neutral
#   -1.0 → net short
NET_BIAS       = 0.0       


# -------------------------
# CORRELATION GENERATOR
# -------------------------
def generate_clustered_corr(n_assets, n_clusters=2, regime="medium", seed=None):
    if seed is not None:
        np.random.seed(seed)

    if regime == "high":
        within_range = (0.6, 0.9)
        between_range = (0.3, 0.6)
    elif regime == "medium":
        within_range = (0.3, 0.6)
        between_range = (0.1, 0.3)
    elif regime == "low":
        within_range = (0.05, 0.2)
        between_range = (0.0, 0.1)
    else:
        raise ValueError("regime must be 'high', 'medium', or 'low'")

    clusters = np.random.choice(n_clusters, size=n_assets)

    corr = np.eye(n_assets)
    for i in range(n_assets):
        for j in range(i+1, n_assets):
            if clusters[i] == clusters[j]:
                rho = np.random.uniform(*within_range)
            else:
                rho = np.random.uniform(*between_range)
            corr[i, j] = corr[j, i] = rho

    eigvals, eigvecs = np.linalg.eigh(corr)
    eigvals[eigvals < 1e-6] = 1e-6
    corr_psd = eigvecs @ np.diag(eigvals) @ eigvecs.T

    d = np.sqrt(np.diag(corr_psd))
    corr_psd = corr_psd / d[:, None] / d[None, :]

    return corr_psd, clusters


# -------------------------
# VOLATILITY GENERATOR
# -------------------------
def assign_vols(n_assets, regime="medium", seed=None):
    if seed is not None:
        np.random.seed(seed)

    if regime == "high":
        vol_range = (0.03, 0.05)
    elif regime == "medium":
        vol_range = (0.01, 0.02)
    elif regime == "low":
        vol_range = (0.005, 0.01)
    else:
        raise ValueError("regime must be 'high', 'medium', or 'low'")

    return np.random.uniform(*vol_range, size=n_assets)


# -------------------------
# SUPPORT FUNCTIONS
# -------------------------
def kupiec_test(exceedances, T, alpha):
    N = exceedances.sum()
    p_hat = N / T
    q = 1 - alpha
    if N == 0 or N == T:
        LR = 0.0
    else:
        LR = -2 * (np.log((1-q)**(T-N) * (q**N)) -
                   np.log((1-p_hat)**(T-N) * (p_hat**N)))
    p_value = 1 - chi2.cdf(LR, 1)
    return N, LR, p_value


def portfolio_weights(t, n, mode="fixed",
                      allow_shorts=True, net_bias=0.0):
    if mode == "fixed":
        np.random.seed(123)
        w = np.random.randn(n) if allow_shorts else np.random.rand(n)
    else:
        base = np.sin(2*np.pi*t/500 + np.arange(n)) + np.random.normal(0, 0.1, n)
        w = base if allow_shorts else np.abs(base)

    # Step 1: normalize gross = 1
    w = w / np.sum(np.abs(w))

    # Step 2: shift toward desired net bias
    current_net = np.sum(w)
    shift = (net_bias - current_net) / n
    w = w + shift

    # Step 3: renormalize gross = 1
    w = w / np.sum(np.abs(w))

    return w


def compute_var(method, sigma, alpha, holding_period=1):
    z = norm.ppf(1 - alpha)
    if method == "normal_simple":
        return -(z * np.sqrt(holding_period) * sigma)
    elif method == "lognormal":
        return -(np.exp(z * np.sqrt(holding_period) * sigma) - 1)
    elif method == "normal_log":
        return -(z * np.sqrt(holding_period) * sigma)
    else:
        raise ValueError("Unknown method")


# -------------------------
# SIMULATION
# -------------------------
np.random.seed(42)
corr_matrix, cluster_labels = generate_clustered_corr(
    n_assets=NUM_ASSETS,
    n_clusters=N_CLUSTERS,
    regime=CORR_REGIME,
    seed=42
)
vols = assign_vols(NUM_ASSETS, regime=VOL_REGIME, seed=123)

cov_matrix = np.diag(vols) @ corr_matrix @ np.diag(vols)
L = np.linalg.cholesky(cov_matrix)
log_returns = np.dot(np.random.randn(NUM_DAYS, NUM_ASSETS), L.T)
log_returns = pd.DataFrame(log_returns)

# -------------------------
# BACKTESTING SUMMARY TABLE
# -------------------------
rows = []

# --- 1-day horizon (no overlap distinction) ---
hp = 1
for method in ["normal_simple", "lognormal", "normal_log"]:
    var_series, ex_series = [], []
    time_range = range(WINDOW + hp - 1, NUM_DAYS)

    for t in time_range:
        w = portfolio_weights(t, NUM_ASSETS, PORTFOLIO_MIX,
                              ALLOW_SHORTS, NET_BIAS)
        data = log_returns.iloc[t-WINDOW:t]

        if WINDOW_TYPE == "ema":
            cov_t = data.ewm(span=WINDOW, adjust=False).cov().iloc[-NUM_ASSETS:]
            cov_t = cov_t.values.reshape(NUM_ASSETS, NUM_ASSETS)
        else:
            cov_t = np.cov(data.values.T)

        sigma_p = np.sqrt(w @ cov_t @ w)

        # 1-day realized simple return
        r_hp = log_returns.iloc[t] @ w
        pnl = np.exp(r_hp) - 1

        # 1-day VaR
        var = compute_var(method, sigma_p, CONF_LEVEL, hp)

        # convert Normal-Log VaR to simple return units
        var_simple = np.exp(var) - 1 if method == "normal_log" else var

        var_series.append(var_simple)
        ex_series.append(int(pnl < -var_simple))

    T = len(var_series)
    expected = (1 - CONF_LEVEL) * T
    actual, LR, p_val = kupiec_test(np.array(ex_series), T, CONF_LEVEL)
    rate = actual / T
    avg_var = np.mean(var_series)

    rows.append({
        "Horizon": "1-day",
        "Method": method,
        "Overlap": "N/A",
        "Sample Size": T,
        "Expected Exc.": round(expected, 2),
        "Actual Exc.": actual,
        "Rate": round(rate, 4),
        "Avg VaR": round(avg_var, 4),
        "LR Stat": round(LR, 3),
        "Kupiec p-val": round(p_val, 3)
    })


# --- 10-day horizon (overlapping and non-overlapping) ---
hp = HOLDING_PERIOD
for overlap_flag in [True, False]:
    for method in ["normal_simple", "lognormal", "normal_log"]:
        var_series, ex_series = [], []

        if overlap_flag:
            time_range = range(WINDOW + hp - 1, NUM_DAYS)
        else:
            time_range = range(WINDOW + hp - 1, NUM_DAYS, hp)

        for t in time_range:
            w = portfolio_weights(t, NUM_ASSETS, PORTFOLIO_MIX,
                                  ALLOW_SHORTS, NET_BIAS)
            data = log_returns.iloc[t-WINDOW:t]

            if WINDOW_TYPE == "ema":
                cov_t = data.ewm(span=WINDOW, adjust=False).cov().iloc[-NUM_ASSETS:]
                cov_t = cov_t.values.reshape(NUM_ASSETS, NUM_ASSETS)
            else:
                cov_t = np.cov(data.values.T)

            sigma_p = np.sqrt(w @ cov_t @ w)

            # 10-day realized simple return
            r_hp = log_returns.iloc[t-hp+1:t+1] @ w
            cum_log = r_hp.sum()
            pnl = np.exp(cum_log) - 1

            # 10-day VaR
            var = compute_var(method, sigma_p, CONF_LEVEL, hp)

            # convert Normal-Log VaR to simple return units
            var_simple = np.exp(var) - 1 if method == "normal_log" else var

            var_series.append(var_simple)
            ex_series.append(int(pnl < -var_simple))

        T = len(var_series)
        expected = (1 - CONF_LEVEL) * T
        actual, LR, p_val = kupiec_test(np.array(ex_series), T, CONF_LEVEL)
        rate = actual / T
        avg_var = np.mean(var_series)

        rows.append({
            "Horizon": "10-day",
            "Method": method,
            "Overlap": "Yes" if overlap_flag else "No",
            "Sample Size": T,
            "Expected Exc.": round(expected, 2),
            "Actual Exc.": actual,
            "Rate": round(rate, 4),
            "Avg VaR": round(avg_var, 4),
            "LR Stat": round(LR, 3),
            "Kupiec p-val": round(p_val, 3)
        })


# Build dataframe
summary_df = pd.DataFrame(rows)

print("\n=== BACKTESTING SUMMARY ===")

print("\n--- 1-day horizon ---")
print(summary_df[summary_df["Horizon"]=="1-day"].drop(columns="Horizon").to_string(index=False))

print("\n--- 10-day horizon (overlapping) ---")
print(summary_df[(summary_df["Horizon"]=="10-day") & (summary_df["Overlap"]=="Yes")].drop(columns=["Horizon","Overlap"]).to_string(index=False))

print("\n--- 10-day horizon (non-overlapping) ---")
print(summary_df[(summary_df["Horizon"]=="10-day") & (summary_df["Overlap"]=="No")].drop(columns=["Horizon","Overlap"]).to_string(index=False))


=== BACKTESTING SUMMARY ===

--- 1-day horizon ---
       Method Overlap  Sample Size  Expected Exc.  Actual Exc.   Rate  Avg VaR  LR Stat  Kupiec p-val
normal_simple     N/A         9750           97.5           95 0.0097   0.0148    0.065         0.798
    lognormal     N/A         9750           97.5           98 0.0101   0.0147    0.003         0.959
   normal_log     N/A         9750           97.5           91 0.0093   0.0149    0.448         0.503

--- 10-day horizon (overlapping) ---
       Method  Sample Size  Expected Exc.  Actual Exc.   Rate  Avg VaR  LR Stat  Kupiec p-val
normal_simple         9741          97.41           88 0.0090   0.0467    0.949         0.330
    lognormal         9741          97.41          107 0.0110   0.0457    0.924         0.336
   normal_log         9741          97.41           79 0.0081   0.0478    3.757         0.053

--- 10-day horizon (non-overlapping) ---
       Method  Sample Size  Expected Exc.  Actual Exc.   Rate  Avg VaR  LR Stat  Kup